<a href="https://colab.research.google.com/github/philosophynote/machine_learning/blob/main/search_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade openai
!pip install google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import sys
import openai
import json
import requests
import webbrowser
from time import sleep
from googleapiclient.discovery import build




def get_recommended_restaurant_info(location):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {
                "role": "system",
                "content": 'JSONフォーマットで答えてください。{"latitude": {float number}, "longitude": {float number}}',
            },
            {
                "role": "user",
                "content": f"{location}の緯度経度は？",
            },
        ],
    )

    message = response["choices"][0]["message"]
    json_data = json.loads(message["content"])
    latitude = json_data["latitude"]
    longitude = json_data["longitude"]

    service = build("customsearch", "v1", developerKey=os.environ["GOOGLE_API_KEY"])
    page_limit = 10
    start_index = 1
    response_google = []
    for n_page in range(0, page_limit):
        try:
            sleep(1)
            response_google.append(service.cse().list(
                q=f"緯度：{latitude}、経度：{longitude} 有名なレストラン",
                cx=os.environ["GOOGLE_CSE_ID"],
                lr='lang_ja',
                num=10,
                start=start_index
            ).execute())
            start_index = response_google[n_page].get("queries").get("nextPage")[0].get("startIndex")
        except Exception as e:
            print(e)
            break
    print(response_google)
    # data = response.json()

    # recommended_restaurant = data["results"]["shop"][0]

    # restaurant_info = {
    #     "name": recommended_restaurant["name"],
    #     "address": recommended_restaurant["address"],
    #     "genre": recommended_restaurant["genre"]["name"],
    #     "budget": recommended_restaurant["budget"]["average"],
    #     "access": recommended_restaurant["access"],
    # }
    # print(restaurant_info)
    # return json.dumps(restaurant_info)

def run_chat_model():
    if len(sys.argv) > 1:
        location = sys.argv[1]
    else:
        location = "東京"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {
                "role": "system",
                "content": "日本語で正確にお店の名前と住所、予算などを紹介してください。"
            },
            {
                "role": "user",
                "content": f"{location}のオススメのお店は？"
            },
        ],
        functions=[
            {
                "name": "get_recommended_restaurant_info",
                "description": "指定された場所に基づいて推奨されるレストランを取得",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都市と地域を指定。例：東京、日本",
                        },
                        "genre": {
                            "type": "string",
                            "description": "レストランのジャンルを指定。例：イタリアン、寿司",
                        },
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    print(response)
    # message = response["choices"][0]["message"]
    # print(message)

    # if message.get("function_call"):
    #     function_name = message["function_call"]["name"]
    #     arguments = json.loads(message["function_call"]["arguments"])
    #     print(arguments)

    #     if arguments["location"]:
    #         function_response = get_recommended_restaurant_info(
    #             location=arguments["location"],
    #          )

    #         second_response = openai.ChatCompletion.create(
    #             model="gpt-3.5-turbo-0613",
    #             messages=[
    #                 {
    #                     "role": "system",
    #                     "content": "日本語で正確にお店の名前と住所、予算などを楽しそうに紹介してください。",
    #                 },
    #                 {"role": "user", "content": f"{location}の近くでオススメのお店は？"},
    #                 message,
    #                 {
    #                     "role": "function",
    #                     "name": function_name,
    #                     "content": function_response,
    #                 },
    #             ],
    #         )

    #         message = second_response["choices"][0]["message"]
    #         print(message["content"])
    #         return second_response

if __name__ == "__main__":
    run_chat_model()


{
  "id": "chatcmpl-7UpuBQNmAGv3fMLKXh6kGQhjxqxAY",
  "object": "chat.completion",
  "created": 1687583719,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_recommended_restaurant_info",
          "arguments": "{\n  \"location\": \"\u6771\u4eac\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 162,
    "completion_tokens": 21,
    "total_tokens": 183
  }
}
